In [42]:
import pandas as pd
import numpy as np
import wfdb
import neurokit2 as nk
from  scipy.signal import hilbert, find_peaks
import csv

In [43]:
hrv_dataset = pd.read_csv('ECG_Cardiac_Features.csv')
directory = 'physionet.org/files/ptb-xl/1.0.3/'
dataset = pd.read_csv('patient_scp.csv')

In [ ]:
csv_file = open('ECG_Hilbert_Features.csv', 'w')
csv_writer = csv.writer(csv_file)
csv_writer.writerow([
    'ecg_id', 
    'peak_amplitude_mean', 'peak_amplitude_max', 'peak_amplitude_std',
    'mean_amplitude_mean', 'mean_amplitude_max', 'mean_amplitude_std',
    'std_amplitude_mean', 'std_amplitude_max', 'std_amplitude_std',
    'qrs_amplitude_mean', 'qrs_amplitude_max', 'qrs_amplitude_std'
])

for index, row in hrv_dataset.iterrows():
    print(f'Processing {index} of {len(hrv_dataset)}')
    ecg_id = row['ecg_id']
    filename_hr = dataset[dataset['ecg_id'] == ecg_id]['filename_hr'].values[0]
    record = wfdb.rdrecord(directory + filename_hr)
    signal = record.p_signal[:, 0]
    ecg_cleaned = signal

    analytic_signal = hilbert(ecg_cleaned)

    amplitude_envelope = np.abs(analytic_signal)

    cg_peaks, ecg_info = nk.ecg_peaks(ecg_cleaned, sampling_rate=500)
    rpeaks = ecg_info["ECG_R_Peaks"]

    peak_amplitudes = []
    mean_amplitudes = []
    std_amplitudes = []
    qrs_amplitudes = []
    for i in range(len(rpeaks) - 1):
        # Segment the amplitude envelope for the current cardiac cycle
        segment = amplitude_envelope[rpeaks[i]:rpeaks[i + 1]]
        
        peak_amplitude = np.max(segment)
        mean_amplitude = np.mean(segment)
        std_amplitude = np.std(segment)

        qrs_start = max(0, rpeaks[i] - 50)
        qrs_end = min(len(amplitude_envelope), rpeaks[i] + 50)
        qrs_amplitude = np.mean(amplitude_envelope[qrs_start:qrs_end])

        peak_amplitudes.append(peak_amplitude)
        mean_amplitudes.append(mean_amplitude)
        std_amplitudes.append(std_amplitude)
        qrs_amplitudes.append(qrs_amplitude)

    try:
        peak_amplitude_mean = np.nan
        peak_amplitude_mean = np.mean(peak_amplitudes)

        peak_amplitude_max = np.nan
        peak_amplitude_max = np.max(peak_amplitudes)

        peak_amplitude_std = np.nan
        peak_amplitude_std = np.std(peak_amplitudes)

        mean_amplitude_mean = np.nan
        mean_amplitude_mean = np.mean(mean_amplitudes)

        mean_amplitude_max = np.nan
        mean_amplitude_max = np.max(mean_amplitudes)

        mean_amplitude_std = np.nan
        mean_amplitude_std = np.std(mean_amplitudes)

        std_amplitude_mean = np.nan
        std_amplitude_mean = np.mean(std_amplitudes)

        std_amplitude_max = np.nan
        std_amplitude_max = np.max(std_amplitudes)

        std_amplitude_std = np.nan
        std_amplitude_std = np.std(std_amplitudes)

        qrs_amplitude_mean = np.nan
        qrs_amplitude_mean = np.mean(qrs_amplitudes)

        qrs_amplitude_max = np.nan
        qrs_amplitude_max = np.max(qrs_amplitudes)

        qrs_amplitude_std = np.nan
        qrs_amplitude_std = np.std(qrs_amplitudes)


    except Exception as e:
        print(e)
    
    finally:
        csv_writer.writerow([
            ecg_id, 
            peak_amplitude_mean, peak_amplitude_max, peak_amplitude_std,
            mean_amplitude_mean, mean_amplitude_max, mean_amplitude_std,
            std_amplitude_mean, std_amplitude_max, std_amplitude_std,
            qrs_amplitude_mean, qrs_amplitude_max, qrs_amplitude_std
        ])

In [45]:
hilbert_dataset = pd.read_csv('ECG_Hilbert_Features.csv')

hilbert_dataset['Phase Locking Value'] = np.nan
hilbert_dataset['Phase Duration'] = np.nan

for index, row in hilbert_dataset.iterrows():
    print(f'Processing {index + 1} of {len(hilbert_dataset)}')
    
    ecg_id = row['ecg_id']
    filename_hr = dataset[dataset['ecg_id'] == ecg_id]['filename_hr'].values[0]
    
    record = wfdb.rdrecord(directory + filename_hr)
    signal = record.p_signal[:, 0]


    ecg_cleaned = signal
    
    try:
        analytic_signal = hilbert(ecg_cleaned)
        instantaneous_phase = np.unwrap(np.angle(analytic_signal))

        rpeaks, _ = nk.ecg_peaks(ecg_cleaned, sampling_rate=500)
        rpeaks = rpeaks["ECG_R_Peaks"]
        
        phase_rpeaks = instantaneous_phase[rpeaks]
        plv = np.abs(np.mean(np.exp(1j * (phase_rpeaks - np.mean(phase_rpeaks)))))
        
        phase_diff = np.diff(instantaneous_phase)
        phase_duration = np.sum(phase_diff > 0) / 500  # Convert to seconds

        hilbert_dataset.loc[index, 'Phase Locking Value'] = plv
        hilbert_dataset.loc[index, 'Phase Duration'] = phase_duration
    except Exception as e:
        print(f"Error processing phase features for {ecg_id}: {e}")
        hilbert_dataset.loc[index, 'Phase Locking Value'] = np.nan
        hilbert_dataset.loc[index, 'Phase Duration'] = np.nan

# Save the updated dataset
hilbert_dataset.to_csv('ECG_Hilbert_Features_Updated.csv', index=False)

Processing 1 of 20773
Processing 2 of 20773
Processing 3 of 20773
Processing 4 of 20773
Processing 5 of 20773
Processing 6 of 20773
Processing 7 of 20773
Processing 8 of 20773
Processing 9 of 20773
Processing 10 of 20773
Processing 11 of 20773
Processing 12 of 20773
Processing 13 of 20773
Processing 14 of 20773
Processing 15 of 20773
Processing 16 of 20773
Processing 17 of 20773
Processing 18 of 20773
Processing 19 of 20773
Processing 20 of 20773
Processing 21 of 20773
Processing 22 of 20773
Processing 23 of 20773
Processing 24 of 20773
Processing 25 of 20773
Processing 26 of 20773
Processing 27 of 20773
Processing 28 of 20773
Processing 29 of 20773
Processing 30 of 20773
Processing 31 of 20773
Processing 32 of 20773
Processing 33 of 20773
Processing 34 of 20773
Processing 35 of 20773
Processing 36 of 20773
Processing 37 of 20773
Processing 38 of 20773
Processing 39 of 20773
Processing 40 of 20773
Processing 41 of 20773
Processing 42 of 20773
Processing 43 of 20773
Processing 44 of 207